### TRAN CHI TOAN - toanchitran2302@gmail.com

In [1]:
from datetime import date
today = date.today().isoformat()
from bs4 import BeautifulSoup 
import requests
import re 
from time import sleep
import pandas as pd
from datetime import date

def monster (job_query = None, location_query=''):
    
    
    if job_query is None:
        job_query = 'data-analytics'
    else:
        job_query = job_query.split()
        job_query= '-'.join(w for w in job_query)
    
    if location_query is not None:
        location_query = location_query.split()
        location_query = '-'.join(w for w in location_query)
        
        
    #Get job detail job description fuction
    def job(url):
        job_page = requests.get(url)
        job_soup = BeautifulSoup(job_page.text, 'lxml')
        job_desc = job_soup.find('div', attrs={'id':'JobDescription'}).text
        return job_desc

    # Clean text
    def clean_text(text):
        lines = (line.strip() for line in text.splitlines()) # break into lines
        
        chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
        def chunk_space(chunk):
            chunk_out = chunk + ' ' # Need to fix spacing issue
            return chunk_out  
        
    
        text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
        
        
        # Now clean out all of the unicode junk (this line works great!!!)
        
        try:
            text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
        except:                                                            # in a way that this works, can occasionally throw
            return                                                         # an exception
        return text



    # Scrapt the first page to find the number of jobs 
    base_URL= ''.join(['https://www.monster.com/jobs/search/?q='+job_query+'&where='+location_query])
    base_page = requests.get(base_URL)
    base_soup = BeautifulSoup(base_page.text, 'html.parser')
    number_job = sum(int(i) for i in re.findall('\d+', base_soup.find('h2', attrs={'class':'figure'}).text))

    page_num = int(number_job/20)

    #Scrapt all the job
    crawl_page = ''.join([base_URL, '&page=', str(page_num)])
    page = requests.get(crawl_page)
    soup = BeautifulSoup(page.text, 'html.parser')

    # We need those dataframe to store information on company, job title, link, location and job description
    company = pd.DataFrame(columns=['Company'])
    location = pd.DataFrame(columns=['Location'])
    title = pd.DataFrame(columns=['Title'])
    URL = pd.DataFrame(columns=['URL'])
    job_description = pd.DataFrame(columns=['Job Description'])
    final_query= pd.DataFrame(columns=['Job Query'])
    links = []
    
    #Append Job Query
    

    # Find companies
    companies = soup.findAll(name='div', attrs={'class':'company'})

    # Find job titles
    titles = soup.findAll(name='h2', attrs={'class':'title'})

    # Find job locations
    locations = soup.findAll(name='div', attrs={'class':'location'})

    # Append all companies into companies dataframe
    for a in companies:
        comp=a.text.strip()
        company= company.append({'Company':comp},ignore_index=True)

    # Append all job locations to locations dataframe
    for b in locations:
        loc = b.text.strip()
        if loc != 'Location\n\nx':
            location= location.append({'Location':loc}, ignore_index=True)


    for c in titles:
        link = c.find('a').attrs['href']
        links.append(link)
        title = title.append({'Title': c.text.strip()}, ignore_index=True)
        URL=URL.append({'URL':link}, ignore_index=True)
        final_query = final_query.append({'Job Query':job_query}, ignore_index=True)
   
    for e in links:
        job_desc_l = job(e)
        job_clean = clean_text(job_desc_l)
        job_clean = job_clean.replace("#jobBodyContent ul { 		margin-bottom: 13px; 		line-height: 16px; 	}", "")
        job_clean = job_clean.replace("#ejb_SubHdTxt, .ejb_SubHdTxt { background-color: #f1f1f1;} 		.addthis_button {font-size:8pt;} div#content{ font-size: 12px; /*font-family: Arial, Helvetica, sans-serif; */ 			line-height: 20px; 			/*width: 758px; */ 			overflow: hidden; 		} 		img#barona_top_banner{ 			width: 100%; 			height: auto; 			margin-bottom: 30px; 		}	 		div#content_text{ 			margin-left: 40px; 			margin-right: 40px; 		} 		div#barona-text-area-padding{ 			padding-bottom: 40px; 		} 		div#content_text a{font-size: 14px; font-weight: bold; color: #0d51b4; text-decoration: none;} div#content_text h1{font-size: 26px; font-weight: bold; margin-bottom: 30px;} 	 		div#content_text h2{font-size: 16px; font-weight: bold; } 		div#content_text span.company-title{font-weight: bold; } div#employer-company-card{ 			border-top: 1px solid #dedede; 			padding-top: 40px; 			padding-bottom: 40px;		 		} 		img#barona-employer-logo{ 			max-width: 185px; 			height: auto; 			margin-bottom: 20px; 		}	 		div#barona-company-card{ 			border-top: 1px solid #dedede; 			padding-top: 40px; 			padding-bottom: 40px;		 		} 		img#barona-logo{ 			max-width: 185px; 			height: auto; 			margin-bottom: 20px; 		} ", "")
        job_description = job_description.append({'Job Description':job_clean}, ignore_index=True)
        sleep(1)

    job=pd.concat([final_query, company, location, title, URL, job_description], axis=1)
    
    # Uncomment line below to create a csv file with the new job found
    #job.to_csv('monster_fi_'+today+'_'+job_query''_'+location_query+'.csv', encoding='utf-8')   
    
    return job